## Import and Configuration

In [66]:
import os
import re
import csv
import json
import math
import openai
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from datetime import datetime
from collections import defaultdict, Counter
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import ListedColormap

In [67]:
def extract_brackets(text, brackets='[]'):
    assert len(brackets) == 2
    pattern = re.escape(brackets[0]) + r'(.*?)' + re.escape(brackets[1])
    matches = re.findall(pattern, text)
    return matches

def extract_amout(
    message, 
    prefix='',
    print_except=True,
    type=float,
    brackets='[]'
):
    try:
        matches = extract_brackets(message, brackets=brackets)
        matches = [s[len(prefix):] \
            if s.startswith(prefix) \
            else s for s in matches]
        invalid = False
        if len(matches) == 0:
            invalid = True
        for i in range(len(matches)):
            if matches[i] != matches[0]:
                invalid = True
        if invalid:
            raise ValueError('Invalid answer: %s' % message)
        return type(matches[0])
    except Exception as e: 
        if print_except: print(e)
        return None

def extract_choices(recrods):
    choices = [extract_amout(
        messages[-1]['content'], 
        prefix='$', 
        print_except=True,
        type=float) for messages in records['messages']
    ]
    choices = [x for x in choices if x is not None]
    # print(choices)
    return choices

In [68]:
def choices_to_df(choices, hue):
    df = pd.DataFrame(choices, columns=['choices'])
    df['hue'] = hue
    df['hue'] = df['hue'].astype(str)
    return df

def plot(
        df, 
        title='',
        x='choices',
        hue='hue',
        binrange=None, 
        binwidth=None,
        stat='count',
        multiple='dodge'
    ):
    if binrange is None:
        binrange = (df[x].min(), df[x].max())
    df = df.dropna(axis=0, subset=[x]).reset_index()
    plt.figure(figsize=(8, 6))
    ax = sns.histplot(
        data=df, 
        x=x,
        hue=hue, 
        kde=True,
        binrange=binrange, 
        binwidth=binwidth,
        stat=stat,
        multiple=multiple,
        shrink=.8,
    )
    ax.set_title(title)
    return ax

In [69]:
def plot_facet(
    df_list,
    x='choices',
    hue='hue',
    palette=None,
    binrange=None,
    bins=10,
    # binwidth=10,
    stat='count',
    x_label='',
    sharex=True,
    sharey=False,
    subplot=sns.histplot,
    xticks_locs=None,
    # kde=False,
    **kwargs
):
    data = pd.concat(df_list)
    if binrange is None:
        binrange = (data[x].min(), data[x].max())
    g = sns.FacetGrid(
        data, row=hue, hue=hue, 
        palette=palette,
        aspect=2, height=2, 
        sharex=sharex, sharey=sharey,
        despine=True,
    )
    g.map_dataframe(
        subplot, 
        x=x, 
        # kde=kde, 
        binrange=binrange, 
        bins=bins,
        stat=stat,
        **kwargs
    )
    # g.add_legend(title='hue')
    g.set_axis_labels(x_label, stat.title())
    g.set_titles(row_template="{row_name}")
    for ax in g.axes.flat:
        ax.yaxis.set_major_formatter(
            FuncFormatter(lambda y, pos: '{:.2f}'.format(y))
        )
    
    binwidth = (binrange[1] - binrange[0]) / bins
    if xticks_locs is None:
        locs = np.linspace(binrange[0], binrange[1], bins//2+1)
        locs = [loc + binwidth for loc in locs]
    else: 
        locs = xticks_locs
    labels = [str(int(loc)) for loc in locs]
    locs = [loc + 0.5*binwidth for loc in locs]
    plt.xticks(locs, labels)
    
    g.set(xlim=binrange)
    return g

In [70]:
sns.set(rc={'figure.figsize':(5,4)})
sns.set_style("ticks")

default_palette = sns.color_palette(None)
blue = default_palette[0]
orange = default_palette[1]
green = default_palette[2]
red = default_palette[3]
purple = default_palette[4]

In [71]:
def plot_occupations(
    df_baseline,
    choices_all,
    binrange=(0,100),
    binwidth=5,
    x_label='$ to give',
    stat='density',
    model='ChatGPT-4',
):
    print('baseline: ', len(df_baseline))
    df_list = []
    for occupation in choices_all:
        choices = choices_all[occupation]
        print(occupation, ':', len(choices))
        df = choices_to_df(choices, hue='{} ({})'.format(model, occupation))
        df_list.append(df)
    g = plot_facet(
        df_list=[df_baseline]+df_list,
        binrange=binrange,
        binwidth=binwidth,
        x_label=x_label,
        stat=stat,
    )
    return g

## Fig. 2: Turing Test

Results may vary due to randomness in the simulation.

In [72]:
# def simulate_Turing(samples_0, samples_1, n_bin=10, lim_a=0, lim_b=100, n_draw=100000):
#     hist_0 = np.histogram(samples_0, bins=n_bin, range=(lim_a, lim_b))[0] / len(samples_0)
#     n_wins = 0
#     n_ties = 0
#     for _ in tqdm(range(n_draw)):
#         try:
#             sample_0 = np.random.choice(samples_0)
#             sample_1 = np.random.choice(samples_1)
#             idx_0 = min(math.floor((sample_0 - lim_a) / (lim_b - lim_a) * n_bin), n_bin-1)
#             idx_1 = min(math.floor((sample_1 - lim_a) / (lim_b - lim_a) * n_bin), n_bin-1)
#             if hist_0[idx_1] > hist_0[idx_0]:
#                 n_wins += 1
#             elif hist_0[idx_1] == hist_0[idx_0]:
#                 n_ties += 1
#         except:
#             continue
#     return n_wins / n_draw, n_ties / n_draw, (n_draw - n_wins - n_ties) / n_draw

In [73]:
# print('human', simulate_Turing(df_dictator_human['choices'], df_dictator_human['choices']))
# print('gpt4', simulate_Turing(df_dictator_human['choices'], df_dictator_gpt4['choices']))
# print('turbo', simulate_Turing(df_dictator_human['choices'], df_dictator_turbo['choices']))

In [74]:
# print('human', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_human['choices']))
# print('gpt4', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_gpt4['choices']))
# print('turbo', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_turbo['choices']))

In [75]:
# print('human', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_human['choices']))
# print('gpt4', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_gpt4['choices']))
# print('turbo', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_turbo['choices']))

In [76]:
# print('human', simulate_Turing(df_trust_1_human['choices'], df_trust_1_human['choices']))
# print('gpt4', simulate_Turing(df_trust_1_human['choices'], df_trust_1_gpt4['choices']))
# print('turbo', simulate_Turing(df_trust_1_human['choices'], df_trust_1_turbo['choices']))

In [77]:
# print('human', simulate_Turing(df_trust_3_human['choices'], df_trust_3_human['choices'], lim_b=150))
# print('gpt4', simulate_Turing(df_trust_3_human['choices'], df_trust_3_gpt4['choices'], lim_b=150))
# print('turbo', simulate_Turing(df_trust_3_human['choices'], df_trust_3_turbo['choices'], lim_b=150))

In [78]:
# # Convert to numeric, coercing errors (invalid parsing will be set as NaN)
# samples_0 = pd.to_numeric(df_PG_human['choices'], errors='coerce')
# samples_1 = pd.to_numeric(df_PG_gpt4['choices'], errors='coerce')

In [79]:
# df_PG_turbo['choices'].dtype

In [80]:
# print(df_PG_turbo['choices'].head())

In [81]:


# nan_count = df_PG_turbo['choices'].isna().sum()
# print(f"Number of NaN values: {nan_count}")

df_PG_turbo['choices'].fillna(0, inplace=True)

In [82]:
# df_PG_turbo['choices'] = df_PG_turbo['choices'].dropna()

In [83]:
# print('human', simulate_Turing(df_PG_human['choices'], df_PG_human['choices'], lim_b=20))
# print('gpt4', simulate_Turing(df_PG_human['choices'], df_PG_gpt4['choices'], lim_b=20))
# print('gpt4', simulate_Turing(df_PG_turbo['choices'], df_PG_turbo['choices'], lim_b=20))

In [84]:
# print('human', simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_human['']))
# print('gpt4', simulate_Turing(prefix_to_choices_human[''],prefix_to_choices_model['ChatGPT-4']['']))
# print('turbo', simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_model['ChatGPT-3']['']))

In [85]:
# print('human', r_coo_human * r_def_human, r_coo_human * r_coo_human + r_def_human * r_def_human, r_def_human * r_coo_human)
# print('gpt4', r_coo_human * r_def_gpt4, r_coo_human * r_coo_gpt4 + r_def_human * r_def_gpt4, r_def_human * r_coo_gpt4)
# print('turbo', r_coo_human * r_def_turbo, r_coo_human * r_coo_turbo + r_def_human * r_def_turbo, r_def_human * r_coo_turbo)

# n_coo_human = 36269
# n_def_human = 44114
# r_coo_human = n_coo_human / (n_coo_human + n_def_human)
# r_def_human = n_def_human / (n_coo_human + n_def_human)
# print(r_coo_human, r_def_human)

# n_coo_gpt4 = 29 + 0 + 0 + 26
# n_def_gpt4 = 0 + 1 + 1 + 3
# n_coo_turbo = 21 + 3 + 7 + 15
# n_def_turbo = 3 + 3 + 4 + 4
# r_coo_gpt4 = n_coo_gpt4 / (n_coo_gpt4 + n_def_gpt4)
# r_def_gpt4 = n_def_gpt4 / (n_coo_gpt4 + n_def_gpt4)
# r_coo_turbo = n_coo_turbo / (n_coo_turbo + n_def_turbo)
# r_def_turbo = n_def_turbo / (n_coo_turbo + n_def_turbo)
# print(r_coo_gpt4, r_def_gpt4)
# print(r_coo_turbo, r_def_turbo)

In [86]:


import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import pandas as pd

# player_average('human', df_dictator_human['choices'])
# player_average('gpt4', df_dictator_gpt4['choices'])
# player_average('turbo', df_dictator_turbo['choices'])

# player_average('human', [1]*n_coo_human + [0]*n_def_human)
# player_average('gpt4', [1]*n_coo_gpt4 + [0]*n_def_gpt4)
# player_average('turbo', [1]*n_coo_turbo + [0]*n_def_turbo)


# Average_human_result = simulate_Turing(df_average_human['choices'], df_average_human['choices'])
# Average_gpt4_result = simulate_Turing(df_average_human['choices'], df_average_gpt4['choices'])
# Average_turbo_result = simulate_Turing(df_average_human['choices'], df_average_turbo['choices'])




# Dictator_human_result = simulate_Turing(df_dictator_human['choices'], df_dictator_human['choices'])
# Dictator_gpt4_result = simulate_Turing(df_dictator_human['choices'], df_dictator_gpt4['choices'])
# Dictator_turbo_result = simulate_Turing(df_dictator_human['choices'], df_dictator_turbo['choices'])

# Ultimatum_1_human_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_human['choices'])
# Ultimatum_1_gpt4_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_gpt4['choices'])
# Ultimatum_1_turbo_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_turbo['choices'])

# Ultimatum_2_human_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_human['choices'])
# Ultimatum_2_gpt4_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_gpt4['choices'])
# Ultimatum_2_turbo_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_turbo['choices'])

# Trust_1_human_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_human['choices'])
# Trust_1_gpt4_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_gpt4['choices'])
# Trust_1_turbo_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_turbo['choices'])

# Trust_3_human_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_human['choices'], lim_b=150)
# Trust_3_gpt4_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_gpt4['choices'], lim_b=150)
# Trust_3_turbo_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_turbo['choices'], lim_b=150)

# Public_Goods_human_result = simulate_Turing(df_PG_human['choices'], df_PG_human['choices'], lim_b=20)
# Public_Goods_gpt4_result = simulate_Turing(df_PG_human['choices'], df_PG_gpt4['choices'], lim_b=20)
# Public_Goods_turbo_result = simulate_Turing(df_PG_turbo['choices'], df_PG_turbo['choices'], lim_b=20)

# Bomb_Risk_human_result = simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_human[''])
# Bomb_Risk_gpt4_result = simulate_Turing(prefix_to_choices_human[''],prefix_to_choices_model['ChatGPT-4'][''])
# Bomb_Risk_turbo_result = simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_model['ChatGPT-3'][''])

# Prisoner_Dilemma_human_result = r_coo_human * r_def_human, r_coo_human * r_coo_human + r_def_human * r_def_human, r_def_human * r_coo_human
# Prisoner_Dilemma_gpt4_result = r_coo_human * r_def_gpt4, r_coo_human * r_coo_gpt4 + r_def_human * r_def_gpt4, r_def_human * r_coo_gpt4
# Prisoner_Dilemma_turbo_result = r_coo_human * r_def_turbo, r_coo_human * r_coo_turbo + r_def_human * r_def_turbo, r_def_human * r_coo_turbo


# data = {
#     # 'Average': {
#     #     'Human': Average_human_result,
#     #     'GPT-4': Average_gpt4_result,
#     #     'Turbo': Average_turbo_result,
#     # },
#     'Dictator': {
#         'Human': Dictator_human_result,
#         'GPT-4': Dictator_gpt4_result,
#         'Turbo': Dictator_turbo_result,
#     },
#     'Ultimatum 1': {
#         'Human': Ultimatum_1_human_result,
#         'GPT-4': Ultimatum_1_gpt4_result,
#         'Turbo': Ultimatum_1_turbo_result,
#     },
#     'Ultimatum 2': {
#         'Human': Ultimatum_2_human_result,
#         'GPT-4': Ultimatum_2_gpt4_result,
#         'Turbo': Ultimatum_2_turbo_result,
#     },
#     'Trust 1': {
#         'Human': Trust_1_human_result,
#         'GPT-4': Trust_1_gpt4_result,
#         'Turbo': Trust_1_turbo_result,
#     },
#     'Trust 3': {
#         'Human': Trust_3_human_result,
#         'GPT-4': Trust_3_gpt4_result,
#         'Turbo': Trust_3_turbo_result,
#     },
#     'Public Goods': {
#         'Human': Public_Goods_human_result,
#         'GPT-4': Public_Goods_gpt4_result,
#         'Turbo': Public_Goods_turbo_result,
#     },
#     'Bomb Risk': {
#         'Human': Bomb_Risk_human_result,
#         'GPT-4': Bomb_Risk_gpt4_result,
#         'Turbo': Bomb_Risk_turbo_result,
#     },
#     'Prisoner‘s Dilemma': {
#         'Human': Prisoner_Dilemma_human_result,
#         'GPT-4': Prisoner_Dilemma_gpt4_result,
#         'Turbo': Prisoner_Dilemma_turbo_result,
#     }
# }

# data = {
    
#     'Average': {
#         'Human': (0.39472, 0.20789, 0.39539),
#         'GPT-4': (0.45068, 0.20923, 0.34009),
#         'Turbo': (0.14191, 0.10243, 0.75566),
#     },
#     'Dictator': {
#         'Human': (0.39672, 0.20789, 0.39539),
#         'GPT-4': (0.45068, 0.20923, 0.34009),
#         'Turbo': (0.14191, 0.10243, 0.75566),
#     },
#     'Ultimatum 1': {
#         'Human': (0.39641, 0.20612, 0.39747),
#         'GPT-4': (0.65658, 0.34342, 0.0),
#         'Turbo': (0.35094, 0.19359, 0.45547),
#     },
#     'Ultimatum 2': {
#         'Human': (0.4103, 0.17962, 0.41008),
#         'GPT-4': (0.38777, 0.18544, 0.42679),
#         'Turbo': (0.23764, 0.12665, 0.63571),
#     },
#     'Trust 1': {
#         'Human': (0.42553, 0.1499, 0.42457),
#         'GPT-4': (0.24883, 0.0994, 0.65177),
#         'Turbo': (0.22511, 0.09314, 0.68175),
#     },
#     'Trust 3': {
#         'Human': (0.41412, 0.17279, 0.41309),
#         'GPT-4': (0.52801, 0.18393, 0.28806),
#         'Turbo': (0.52552, 0.17863, 0.29585),
#     },
#     'Public Goods': {
#         'Human': (0.42576, 0.14121, 0.43303),
#         'GPT-4': (0.63608, 0.20542, 0.1585),
#         'Turbo': (0.0, 1.0, 0.0),
#     },
#     'Bomb Risk': {
#         'Human': (0.43803, 0.12385, 0.43812),
#         'GPT-4': (0.66265, 0.18317, 0.15418),
#         'Turbo': (0.61836, 0.18062, 0.20102),
#     },
#     'Prisoner‘s Dilemma': {
#         'Human': (0.24761879117560937, 0.5047624176487813, 0.24761879117560937),
#         'GPT-4': (0.03760019738833667, 0.4593353072167, 0.5030644953949632),
#         'Turbo': (0.10528055268734268, 0.473974596618688, 0.42074485069396933),
#     }
# }

# # Transform the data into a format suitable for Altair
# records = []
# for scenario, scenario_data in data.items():
#     for entity, values in scenario_data.items():
#         records.append({
#             "scenario": scenario,
#             "entity": entity,
#             "Estimated More likely Human": values[0],
#             "Estimated Equally likely Human/AI": values[1],
#             "Estimated More Likely AI": values[2]
#         })

# # Create a DataFrame
# df = pd.DataFrame.from_records(records)

# # Melt the DataFrame to have category names and percentages in separate columns
# df_melted = df.melt(id_vars=['scenario', 'entity'], 
#                     value_vars=['Estimated More likely Human', 'Estimated Equally likely Human/AI', 'Estimated More Likely AI'],
#                     var_name='category', value_name='percentage')

# # Define the color scale
# color_scale = alt.Scale(domain=['Estimated More likely Human', 'Estimated Equally likely Human/AI', 'Estimated More Likely AI'],
#                         range=['#2ca02c', '#ff7f0e', '#d62728'])

# # Create the selection for the interactive highlight
# highlight = alt.selection_multi(on='mouseover',fields=['entity'], empty='none')
# # alt.selection_single(on='mouseover', fields=['category'], empty='none')

# # Define the base chart with a bar mark and the selection highlight
# base_chart = alt.Chart(df_melted).mark_bar().encode(
#     x=alt.X('sum(percentage)', stack="normalize", title='', axis=alt.Axis(format='.0%')),
#     y=alt.Y('entity:N', title='', sort=alt.EncodingSortField('entity', op='min', order='descending')),
#     color=alt.Color('category', scale=color_scale),
#     opacity=alt.condition(highlight, alt.value(1), alt.value(0.2)),
#     tooltip=[alt.Tooltip('scenario'), alt.Tooltip('entity'), alt.Tooltip('category'), alt.Tooltip('sum(percentage):Q', format='.2%')]
# ).properties(
#     width=450,  # Width of the individual charts
#     height=200
# ).add_selection(
#     highlight
# )

# # Facet the base chart into two rows
# faceted_chart = base_chart.facet(
#     facet=alt.Facet('scenario', title='', header=alt.Header(labelOrient='top', titleOrient='top')),
#     columns=3  # Display four graphs in each column
# )

# # Adjust spacing between the rows for better readability
# faceted_chart = faceted_chart.configure_facet(spacing=10)

# # Show the chart
# faceted_chart



## Fig. 3: Distributions

### Dictator Game

#### Human Data

In [87]:
binrange = (0, 100)
moves = []
with open('data/dictator.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    col2idx = {col: idx for idx, col in enumerate(header)}
    for row in reader:
        record = {col: row[idx] for col, idx in col2idx.items()}

        if record['Role'] != 'first': continue
        if int(record['Round']) > 1: continue
        if int(record['Total']) != 100: continue
        if record['move'] == 'None': continue
        if record['gameType'] != 'dictator': continue

        move = float(record['move'])
        if move < binrange[0] or \
            move > binrange[1]: continue
        
        moves.append(move)

df_dictator_human = choices_to_df(moves, 'Human')

#### Model Data

In [88]:
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_dictator_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [25, 35, 70, 30, 20, 25, 40, 80, 30, 30, 40, 30, 30, 30, 30, 30, 40, 40, 30, 30, 40, 30, 60, 20, 40, 25, 30, 30, 30]
df_dictator_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

In [89]:
print(len(df_dictator_gpt4.dropna()))
print(len(df_dictator_turbo.dropna()))

30
29


In [90]:
records = json.load(open('records/dictator_wo_ex_2023_03_13-11_24_07_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(x) for x in choices]))

Invalid answer: Sure, here's my choice: 

I would divide the money as follows: 
- I would give [$35] to the other player.
- I would keep [$65] for myself. 

This way, the other player still gets a significant amount of money, but I also keep a majority of it.
Invalid answer: Sure, here's how I would divide the $100: 

I would keep [$30] for myself and give the other player [$70].
Invalid answer: Here's one way to divide the $100: 

I will give [$30] to the other player and keep the remaining [$70].
Invalid answer: Sure, here's one way to divide the money:

I would give the other player [$25] and keep [$75] for myself.
Invalid answer: Sure! Here's my choice: 

I would divide the money with $80 for the other player [$80] and $20 for myself [$20]. I believe this is a fair division as the other player will receive a significant amount of money, while I will still receive a portion for my participation.
Invalid answer: Sure! Here's one possible way to divide the money: 

You can keep [$70],

#### Plot

### Ultimatum Game

#### Human Data

In [92]:
df = pd.read_csv('data/ultimatum_strategy.csv')
df = df[df['gameType'] == 'ultimatum_strategy']
df = df[df['Role'] == 'player']
df = df[df['Round'] == 1]
df = df[df['Total'] == 100]
df = df[df['move'] != 'None']
df['propose'] = df['move'].apply(lambda x: eval(x)[0])
df['accept'] = df['move'].apply(lambda x: eval(x)[1])
df = df[(df['propose'] >= 0) & (df['propose'] <= 100)]
df = df[(df['accept'] >= 0) & (df['accept'] <= 100)]
# df.head()

In [93]:
df_ultimatum_1_human = choices_to_df(list(df['propose']), 'Human')
df_ultimatum_2_human = choices_to_df(list(df['accept']), 'Human')

#### Model Data

In [94]:
choices = [50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
df_ultimatum_1_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [40, 40, 40, 30, 70, 70, 50, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 30, 30, 35, 50, 40, 70, 40, 60, 60, 70, 40, 50]
df_ultimatum_1_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 50.0, 25.0, 50.0, 1.0, 1.0, 20.0, 50.0, 50.0, 50.0, 20.0, 50.0, 1.0, 1.0, 1.0, 50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 20.0, 1.0] + [0, 1]
df_ultimatum_2_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [None, 50, 50, 50, 50, 30, None, None, 30, 33.33, 40, None, 50, 40, None, 1, 30, None, 10, 50, 30, 10, 30, None, 30, None, 10, 30, 30, 30]
df_ultimatum_2_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

In [95]:
print(len(df_ultimatum_1_gpt4.dropna()))
print(len(df_ultimatum_1_turbo.dropna()))

print(len(df_ultimatum_2_gpt4.dropna()))
print(len(df_ultimatum_2_turbo.dropna()))

30
30
30
22


#### Plot

#### Reproducibility

In [96]:
choices = [50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 50.0, 25.0, 50.0, 1.0, 1.0, 20.0, 50.0, 50.0, 50.0, 20.0, 50.0, 1.0, 1.0, 1.0, 50.0, 50.0, 50.0, 1.0, 1.0, 1.0, 20.0, 1.0] + [0, 1]
df_ultimatum_2_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4 (0314, 04.05)'))

# records = json.load(open('records/ultimatum_2_gpt4_2023_12_29-10_42_42_PM.json', 'r'))
# choices = extract_choices(records)
# df_ultimatum_2_gpt4_1229 = choices_to_df(choices, hue=str('ChatGPT-4 (0314, 12.29)'))

# records = json.load(open('records/ultimatum_2_gpt4_1106_2023_12_29-11_15_35_PM.json', 'r'))
# choices = extract_choices(records)
# df_ultimatum_2_gpt4_1106_1229 = choices_to_df(choices, hue=str('ChatGPT-4 (1106, 12.29)'))

# plot_facet(
#     df_list=[
#         df_ultimatum_2_gpt4,
#         df_ultimatum_2_gpt4_1229,
#         df_ultimatum_2_gpt4_1106_1229,
#     ],
#     binrange=(0, 100),
#     # binwidth=10,
#     stat='density',
#     x_label='Minimum proposal to accept ($)',
# )
# plt.savefig('figures/repro-ultimatum-respond.pdf', format='pdf', bbox_inches='tight')
# plt.show()

In [97]:
choices = [None, 50, 50, 50, 50, 30, None, None, 30, 33.33, 40, None, 50, 40, None, 1, 30, None, 10, 50, 30, 10, 30, None, 30, None, 10, 30, 30, 30]
df_ultimatum_2_turbo = choices_to_df(choices, hue=str('ChatGPT-3 (0301, 03.13)'))

# records = json.load(open('records/ultimatum_2_turbo_2023_12_29-10_43_41_PM.json', 'r'))
# choices = extract_choices(records)
# df_ultimatum_2_turbo_1229 = choices_to_df(choices, hue=str('ChatGPT-3 (0301, 12.29)'))

# records = json.load(open('records/ultimatum_2_turbo_1106_2023_12_29-11_57_30_PM.json', 'r'))
# choices = extract_choices(records)
# df_ultimatum_2_turbo_1106_1229 = choices_to_df(choices, hue=str('ChatGPT-3 (1106, 12.29)'))

# plot_facet(
#     df_list=[
#         df_ultimatum_2_turbo,
#         df_ultimatum_2_turbo_1229,
#         df_ultimatum_2_turbo_1106_1229,
#     ],
#     binrange=(0, 100),
#     # binwidth=10,
#     stat='density',
#     x_label='Minimum proposal to accept ($)',
# )
# # plt.savefig('figures/cmp-ultimatum-respond.pdf', format='pdf', bbox_inches='tight')
# plt.show()

### Trust Game

#### Human Data

In [98]:
binrange = (0, 100)
moves_1 = []
moves_2 = defaultdict(list)
with open('data/trust_investment.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)
    col2idx = {col: idx for idx, col in enumerate(header)}
    for row in reader:
        record = {col: row[idx] for col, idx in col2idx.items()}

        # if record['Role'] != 'first': continue
        if int(record['Round']) > 1: continue
        # if int(record['Total']) != 100: continue
        if record['move'] == 'None': continue
        if record['gameType'] != 'trust_investment': continue

        if record['Role'] == 'first':
            move = float(record['move'])
            if move < binrange[0] or \
                move > binrange[1]: continue
            moves_1.append(move)
        elif record['Role'] == 'second':
            inv, ret = eval(record['roundResult'])
            if ret < 0 or \
                ret > inv * 3: continue
            moves_2[inv].append(ret)
        else: continue

df_trust_1_human = choices_to_df(moves_1, 'Human')
df_trust_2_human = choices_to_df(moves_2[10], 'Human')
df_trust_3_human = choices_to_df(moves_2[50], 'Human')
df_trust_4_human = choices_to_df(moves_2[100], 'Human')

#### Model Data

In [99]:
choices = [50.0, 50.0, 40.0, 30.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0, 50.0, 30.0, 30.0, 50.0, 50.0, 50.0, 40.0, 40.0, 50.0, 50.0, 50.0, 50.0, 40.0, 50.0, 50.0, 50.0, 50.0] 
df_trust_1_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [50.0, 50.0, 30.0, 30.0, 30.0, 60.0, 50.0, 40.0, 20.0, 20.0, 50.0, 40.0, 30.0, 20.0, 30.0, 20.0, 30.0, 60.0, 50.0, 30.0, 50.0, 20.0, 20.0, 30.0, 50.0, 30.0, 30.0, 50.0, 40.0] + [30]
df_trust_1_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.0, 20.0, 15.0]
df_trust_2_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 25.0, 30.0, 30.0, 20.0, 25.0, 30.0, 20.0, 20.0, 18.0] + [20, 20, 20, 25, 25, 25, 30]
df_trust_2_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [100.0, 75.0, 75.0, 75.0, 75.0, 75.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0, 75.0, 75.0, 100.0, 100.0, 100.0, 75.0, 100.0, 100.0, 100.0, 100.0, 75.0, 100.0, 75.0]
df_trust_3_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [150.0, 100.0, 150.0, 150.0, 50.0, 150.0, 100.0, 150.0, 100.0, 100.0, 100.0, 150.0] + [100, 100, 100, 100, 100, 100, 100, 100]
df_trust_3_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

choices = [200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0]
df_trust_4_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))

choices = [225.0, 225.0, 300.0, 300.0, 220.0, 300.0, 250.0] + [200, 200, 250, 200, 200]
df_trust_4_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))

In [100]:
records = json.load(open('records/trust_2_gpt4_2023_04_07-11_46_45_PM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(c) for c in choices]))

20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 20.0, 20.0, 20.0, 20.0, 20.0, 15.0, 15.0, 20.0, 15.0, 15.0, 15.0, 15.0, 15.0, 20.0, 20.0, 15.0


In [101]:
records = json.load(open('records/trust_4_gpt4_2023_04_08-12_24_56_AM.json', 'r'))
choices = extract_choices(records)
print(', '.join([str(c) for c in choices]))

200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 200.0, 200.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0, 150.0


#### Plot

### Public Goods

#### Human Data

In [102]:
df = pd.read_csv('data/public_goods_linear_water.csv')
df = df[df['Role'] == 'contributor']
df = df[df['Round'] <= 3]
df = df[df['Total'] == 20]
df = df[df['groupSize'] == 4]
df = df[df['move'] != None]
df = df[(df['move'] >= 0) & (df['move'] <= 20)]
df = df[df['gameType'] == 'public_goods_linear_water']

In [103]:
round_1 = df[df['Round'] == 1]['move']
round_2 = df[df['Round'] == 2]['move']
round_3 = df[df['Round'] == 3]['move']
print(len(round_1), len(round_2), len(round_3))
df_PG_human = pd.DataFrame({
    'choices': list(round_1)
})
df_PG_human['hue'] = 'Human'
df_PG_human

26686 25911 24122


,choices,hue
0,20.0,Human
1,13.0,Human
2,10.0,Human
3,10.0,Human
4,10.0,Human
...,...,...
26681,0.0,Human
26682,10.0,Human
26683,2.0,Human
26684,0.0,Human


##### Payoffs

#### Model Data

In [159]:
file_names = [
    # 'records/PG_basic_turbo_2023_05_09-02_49_09_AM.json',
    # 'records/PG_basic_turbo_loss_2023_05_09-03_59_49_AM.json'
    'records/PG_basic_gpt4_2023_05_09-11_15_42_PM.json',
    'records/PG_basic_gpt4_loss_2023_05_09-10_44_38_PM.json',
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))
df_PG_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))
df_PG_gpt4.head()
df_PG_turbo.head()

,choices,hue
0,12.0,ChatGPT-3
1,12.0,ChatGPT-3
2,12.0,ChatGPT-3
3,10.0,ChatGPT-3
4,10.0,ChatGPT-3


#### Plot

### Bomb Risk Game

1 safe, 0 bomb

#### Human Data

In [105]:
df = pd.read_csv('data/bomb_risk.csv')
df = df[df['Role'] == 'player']
df = df[df['gameType'] == 'bomb_risk']
df.sort_values(by=['UserID', 'Round'])

prefix_to_choices_human = defaultdict(list)
prefix_to_IPW = defaultdict(list)
prev_user = None
prev_move = None
prefix = ''
bad_user = False
for _, row in df.iterrows():
    if bad_user: continue
    if row['UserID'] != prev_user:
        prev_user = row['UserID']
        prefix = ''
        bad_user = False

    move = row['move']
    if move < 0 or move > 100:
        bad_users = True
        continue
    prefix_to_choices_human[prefix].append(move)

    if len(prefix) == 0:
        prefix_to_IPW[prefix].append(1)
    elif prefix[-1] == '1':
        prev_move = min(prev_move, 98)
        prefix_to_IPW[prefix].append(1./(100 - prev_move))
    elif prefix[-1] == '0':
        prev_move = max(prev_move, 1)
        prefix_to_IPW[prefix].append(1./(prev_move))
    else: assert False
    
    prev_move = move

    prefix += '1' if row['roundResult'] == 'SAFE' else '0'

#### Model Data

In [106]:
prefix_to_choices_model = defaultdict(lambda : defaultdict(list))
for model in ['ChatGPT-4', 'ChatGPT-3']:
    if model == 'ChatGPT-4':
        file_names = [
            'bomb_gpt4_2023_05_15-12_13_51_AM.json'
        ]
    elif model == 'ChatGPT-3':
        file_names = [
            'bomb_turbo_2023_05_14-10_45_50_PM.json'
        ]

    choices = []
    scenarios = []
    for file_name in file_names:
        with open(os.path.join('records', file_name), 'r') as f:
            records = json.load(f)
            choices += records['choices']
            scenarios += records['scenarios']

    assert len(scenarios) == len(choices)
    print('loaded %i valid records' % len(scenarios))

    prefix_to_choice = defaultdict(list)
    prefix_to_result = defaultdict(list)
    prefix_to_pattern = defaultdict(Counter)
    wrong_sum = 0
    for scenarios_tmp, choices_tmp in zip(scenarios, choices):

        result = 0
        for i, scenario in enumerate(scenarios_tmp):
            prefix = tuple(scenarios_tmp[:i])
            prefix = ''.join([str(x) for x in prefix])
            choice = choices_tmp[i]
            
            prefix_to_choice[prefix].append(choice)
            prefix_to_pattern[prefix][tuple(choices_tmp[:-1])] += 1

            prefix = tuple(scenarios_tmp[:i+1])
            if scenario == 1:
                result += choice
            prefix_to_result[prefix].append(result)

    print('# of wrong sum:', wrong_sum)
    print('# of correct sum:', len(scenarios) - wrong_sum)

    prefix_to_choices_model[model] = prefix_to_choice

loaded 80 valid records
# of wrong sum: 0
# of correct sum: 80
loaded 80 valid records
# of wrong sum: 0
# of correct sum: 80


#### Plot

### Prisoner Dilemma

#### Human Data

In [108]:
df = pd.read_csv('data/push_pull.csv')
df = df[df['gameType'] == 'push_pull']
df = df[df['Role'] == 'player']
df = df[(df['move'] == 0) | (df['move'] == 1)]
# df = df[df['Round'] <= 2]
df = df[df['groupSize'] == 2]

counter = -1
playIDs = []
otherMoves = []
for i, row in df.iterrows():
    if row['Round'] == 1:
        counter += 1
    playIDs.append(counter)
    roundResult = eval(row['roundResult'])
    roundResult.remove(row['move'])
    otherMoves.append(roundResult[0])
df['playID'] = playIDs
df['otherMove'] = otherMoves

In [110]:
n_coo_human = 36269
n_def_human = 44114
r_coo_human = n_coo_human / (n_coo_human + n_def_human)
r_def_human = n_def_human / (n_coo_human + n_def_human)
print(r_coo_human, r_def_human)

0.45120236866004004 0.54879763133996


In [113]:
print(counter)

Counter({(1, 1): 10584, (0, 1): 7686, (0, 0): 3821, (1, 0): 3635})


#### Model Data

In [115]:
n_coo_gpt4 = 29 + 0 + 0 + 26
n_def_gpt4 = 0 + 1 + 1 + 3
n_coo_turbo = 21 + 3 + 7 + 15
n_def_turbo = 3 + 3 + 4 + 4
r_coo_gpt4 = n_coo_gpt4 / (n_coo_gpt4 + n_def_gpt4)
r_def_gpt4 = n_def_gpt4 / (n_coo_gpt4 + n_def_gpt4)
r_coo_turbo = n_coo_turbo / (n_coo_turbo + n_def_turbo)
r_def_turbo = n_def_turbo / (n_coo_turbo + n_def_turbo)
print(r_coo_gpt4, r_def_gpt4)
print(r_coo_turbo, r_def_turbo)

0.9166666666666666 0.08333333333333333
0.7666666666666667 0.23333333333333334


In [151]:
file_names = [
    'records/PD_gpt4_two_rounds_push_2023_05_10-10_04_33_PM.json',
    'records/PD_gpt4_two_rounds_pull_2023_05_08-08_57_08_PM.json'
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_gpt4 = choices_to_df(choices, hue=str('ChatGPT-4'))
df_PG_gpt4.head()

,choices,hue
0,Push,ChatGPT-4
1,Push,ChatGPT-4
2,Push,ChatGPT-4
3,Push,ChatGPT-4
4,Push,ChatGPT-4


In [155]:
file_names = [
    'records/PD_turbo_two_rounds_push_2023_05_08-06_03_40_PM.json',
    'records/PD_turbo_two_rounds_pull_2023_05_08-09_23_13_PM.json',
]

choices = []
for file_name in file_names:
    with open(file_name, 'r') as f:
        choices += json.load(f)['choices']
choices_baseline = choices

choices = [tuple(x)[0] for x in choices]
df_PG_turbo = choices_to_df(choices, hue=str('ChatGPT-3'))
df_PG_turbo.head()

,choices,hue
0,Push,ChatGPT-3
1,Push,ChatGPT-3
2,Push,ChatGPT-3
3,Push,ChatGPT-3
4,Push,ChatGPT-3


#### Prisoner's Dilemma

In [119]:
r_coo = r_coo_human
r_def = r_def_human
S = [r_coo * 400 + r_def * 000, r_coo * 700 + r_def * 300]
P = [r_coo * 400 + r_def * 700, r_coo * 000 + r_def * 300]
k = {}
for model in ['Human', 'ChatGPT-4', 'ChatGPT-3']:    
    if model == 'Human':
        n_coo = n_coo_human
        n_def = n_def_human
    elif model == 'ChatGPT-4':
        n_coo = n_coo_gpt4
        n_def = n_def_gpt4
    elif model == 'ChatGPT-3':
        n_coo = n_coo_turbo
        n_def = n_def_turbo
    k[model] = [0] * n_coo + [1] * n_def
    print(model)
    # # estimate_beta(S, P, k[model])
    # estimate_beta(S, P, k[model], r=.5)

Human
ChatGPT-4
ChatGPT-3


### Occupation

### Investment (Trust 2-4)

#### Session 2

### Trust Game

## Fig. 2: Turing Test

Results may vary due to randomness in the simulation.

In [120]:
def simulate_Turing(samples_0, samples_1, n_bin=10, lim_a=0, lim_b=100, n_draw=100000):
    hist_0 = np.histogram(samples_0, bins=n_bin, range=(lim_a, lim_b))[0] / len(samples_0)
    n_wins = 0
    n_ties = 0
    for _ in tqdm(range(n_draw)):
        try:
            sample_0 = np.random.choice(samples_0)
            sample_1 = np.random.choice(samples_1)
            idx_0 = min(math.floor((sample_0 - lim_a) / (lim_b - lim_a) * n_bin), n_bin-1)
            idx_1 = min(math.floor((sample_1 - lim_a) / (lim_b - lim_a) * n_bin), n_bin-1)
            if hist_0[idx_1] > hist_0[idx_0]:
                n_wins += 1
            elif hist_0[idx_1] == hist_0[idx_0]:
                n_ties += 1
        except:
            continue
    return n_wins / n_draw, n_ties / n_draw, (n_draw - n_wins - n_ties) / n_draw

In [121]:
print('human', simulate_Turing(df_dictator_human['choices'], df_dictator_human['choices']))
print('gpt4', simulate_Turing(df_dictator_human['choices'], df_dictator_gpt4['choices']))
print('turbo', simulate_Turing(df_dictator_human['choices'], df_dictator_turbo['choices']))

100%|██████████| 100000/100000 [00:02<00:00, 41561.43it/s]


human (0.39281, 0.21017, 0.39702)


100%|██████████| 100000/100000 [00:02<00:00, 42331.44it/s]


gpt4 (0.4529, 0.2067, 0.3404)


100%|██████████| 100000/100000 [00:02<00:00, 41831.94it/s]

turbo (0.1448, 0.10203, 0.75317)


In [122]:
print('human', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_human['choices']))
print('gpt4', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_gpt4['choices']))
print('turbo', simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_turbo['choices']))

100%|██████████| 100000/100000 [00:02<00:00, 41071.22it/s]


human (0.39721, 0.20656, 0.39623)


100%|██████████| 100000/100000 [00:02<00:00, 41262.40it/s]


gpt4 (0.65777, 0.34223, 0.0)


100%|██████████| 100000/100000 [00:02<00:00, 41798.26it/s]

turbo (0.34592, 0.19585, 0.45823)


In [123]:
print('human', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_human['choices']))
print('gpt4', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_gpt4['choices']))
print('turbo', simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_turbo['choices']))

100%|██████████| 100000/100000 [00:02<00:00, 40985.60it/s]


human (0.40989, 0.18031, 0.4098)


100%|██████████| 100000/100000 [00:02<00:00, 41846.80it/s]


gpt4 (0.38805, 0.18535, 0.4266)


100%|██████████| 100000/100000 [00:02<00:00, 40884.91it/s]


turbo (0.2389, 0.12527, 0.63583)


In [124]:
print('human', simulate_Turing(df_trust_1_human['choices'], df_trust_1_human['choices']))
print('gpt4', simulate_Turing(df_trust_1_human['choices'], df_trust_1_gpt4['choices']))
print('turbo', simulate_Turing(df_trust_1_human['choices'], df_trust_1_turbo['choices']))

100%|██████████| 100000/100000 [00:02<00:00, 41925.73it/s]


human (0.42409, 0.15061, 0.4253)


100%|██████████| 100000/100000 [00:02<00:00, 40608.66it/s]


gpt4 (0.24985, 0.10034, 0.64981)


100%|██████████| 100000/100000 [00:02<00:00, 40726.20it/s]

turbo (0.22239, 0.09342, 0.68419)


In [125]:
print('human', simulate_Turing(df_trust_3_human['choices'], df_trust_3_human['choices'], lim_b=150))
print('gpt4', simulate_Turing(df_trust_3_human['choices'], df_trust_3_gpt4['choices'], lim_b=150))
print('turbo', simulate_Turing(df_trust_3_human['choices'], df_trust_3_turbo['choices'], lim_b=150))

100%|██████████| 100000/100000 [00:02<00:00, 41102.28it/s]


human (0.41664, 0.17169, 0.41167)


100%|██████████| 100000/100000 [00:02<00:00, 40600.07it/s]


gpt4 (0.52819, 0.18393, 0.28788)


100%|██████████| 100000/100000 [00:02<00:00, 41770.41it/s]

turbo (0.52483, 0.18012, 0.29505)


In [138]:
# # Convert to numeric, coercing errors (invalid parsing will be set as NaN)
# samples_0 = pd.to_numeric(df_PG_humapt4['choices'], errors='coerce')n['choices'], errors='coerce')
# samples_1 = pd.to_numeric(df_PG_g

In [134]:
# df_PG_turbo['choices'].fillna(0, inplace=True)


In [157]:
# df_PG_turbo['choices'] = pd.to_numeric(df_PG_turbo['choices'], errors='coerce')

In [161]:
print (np.unique(df_PG_turbo['choices']))

[10. 12.]


In [162]:
print('human', simulate_Turing(df_PG_human['choices'], df_PG_human['choices'], lim_b=20))
print('gpt4', simulate_Turing(df_PG_human['choices'], df_PG_gpt4['choices'], lim_b=20))
print('gpt4', simulate_Turing(df_PG_turbo['choices'], df_PG_turbo['choices'], lim_b=20))

100%|██████████| 100000/100000 [00:02<00:00, 41144.45it/s]


human (0.43118, 0.14051, 0.42831)


100%|██████████| 100000/100000 [00:02<00:00, 41599.45it/s]


gpt4 (0.63632, 0.20653, 0.15715)


100%|██████████| 100000/100000 [00:02<00:00, 41232.83it/s]

gpt4 (0.13817, 0.72381, 0.13802)


In [140]:
print('human', simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_human['']))
print('gpt4', simulate_Turing(prefix_to_choices_human[''],prefix_to_choices_model['ChatGPT-4']['']))
print('turbo', simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_model['ChatGPT-3']['']))

100%|██████████| 100000/100000 [02:40<00:00, 624.19it/s]


human (0.43631, 0.12672, 0.43697)


100%|██████████| 100000/100000 [01:21<00:00, 1232.67it/s]


gpt4 (0.66277, 0.18436, 0.15287)


100%|██████████| 100000/100000 [01:23<00:00, 1200.31it/s]

turbo (0.61797, 0.17785, 0.20418)


In [142]:
print('human', r_coo_human * r_def_human, r_coo_human * r_coo_human + r_def_human * r_def_human, r_def_human * r_coo_human)
print('gpt4', r_coo_human * r_def_gpt4, r_coo_human * r_coo_gpt4 + r_def_human * r_def_gpt4, r_def_human * r_coo_gpt4)
print('turbo', r_coo_human * r_def_turbo, r_coo_human * r_coo_turbo + r_def_human * r_def_turbo, r_def_human * r_coo_turbo)

human 0.24761879117560937 0.5047624176487813 0.24761879117560937
gpt4 0.03760019738833667 0.4593353072167 0.5030644953949632
turbo 0.10528055268734268 0.473974596618688 0.42074485069396933


In [168]:
# data = {

#     'Average': {
#         'Human': (0.39472, 0.20789, 0.39539),
#         'GPT-4': (0.45068, 0.20923, 0.34009),
#         'Turbo': (0.14191, 0.10243, 0.75566),
#     },
#     'Dictator': {
#         'Human': (0.39672, 0.20789, 0.39539),
#         'GPT-4': (0.45068, 0.20923, 0.34009),
#         'Turbo': (0.14191, 0.10243, 0.75566),
#     },
#     'Ultimatum 1': {
#         'Human': (0.39641, 0.20612, 0.39747),
#         'GPT-4': (0.65658, 0.34342, 0.0),
#         'Turbo': (0.35094, 0.19359, 0.45547),
#     },
#     'Ultimatum 2': {
#         'Human': (0.4103, 0.17962, 0.41008),
#         'GPT-4': (0.38777, 0.18544, 0.42679),
#         'Turbo': (0.23764, 0.12665, 0.63571),
#     },
#     'Trust 1': {
#         'Human': (0.42553, 0.1499, 0.42457),
#         'GPT-4': (0.24883, 0.0994, 0.65177),
#         'Turbo': (0.22511, 0.09314, 0.68175),
#     },
#     'Trust 3': {
#         'Human': (0.41412, 0.17279, 0.41309),
#         'GPT-4': (0.52801, 0.18393, 0.28806),
#         'Turbo': (0.52552, 0.17863, 0.29585),
#     },
#     'Public Goods': {
#         'Human': (0.42576, 0.14121, 0.43303),
#         'GPT-4': (0.63608, 0.20542, 0.1585),
#         'Turbo': (0.0, 1.0, 0.0),
#     },
#     'Bomb Risk': {
#         'Human': (0.43803, 0.12385, 0.43812),
#         'GPT-4': (0.66265, 0.18317, 0.15418),
#         'Turbo': (0.61836, 0.18062, 0.20102),
#     },
#     'Prisoner‘s Dilemma': {
#         'Human': (0.24761879117560937, 0.5047624176487813, 0.24761879117560937),
#         'GPT-4': (0.03760019738833667, 0.4593353072167, 0.5030644953949632),
#         'Turbo': (0.10528055268734268, 0.473974596618688, 0.42074485069396933),
#     }
# }

Dictator_human_result = simulate_Turing(df_dictator_human['choices'], df_dictator_human['choices'])
Dictator_gpt4_result = simulate_Turing(df_dictator_human['choices'], df_dictator_gpt4['choices'])
Dictator_turbo_result = simulate_Turing(df_dictator_human['choices'], df_dictator_turbo['choices'])

Ultimatum_1_human_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_human['choices'])
Ultimatum_1_gpt4_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_gpt4['choices'])
Ultimatum_1_turbo_result = simulate_Turing(df_ultimatum_1_human['choices'], df_ultimatum_1_turbo['choices'])

Ultimatum_2_human_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_human['choices'])
Ultimatum_2_gpt4_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_gpt4['choices'])
Ultimatum_2_turbo_result = simulate_Turing(df_ultimatum_2_human['choices'], df_ultimatum_2_turbo['choices'])

Trust_1_human_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_human['choices'])
Trust_1_gpt4_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_gpt4['choices'])
Trust_1_turbo_result = simulate_Turing(df_trust_1_human['choices'], df_trust_1_turbo['choices'])

Trust_3_human_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_human['choices'], lim_b=150)
Trust_3_gpt4_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_gpt4['choices'], lim_b=150)
Trust_3_turbo_result = simulate_Turing(df_trust_3_human['choices'], df_trust_3_turbo['choices'], lim_b=150)

Public_Goods_human_result = simulate_Turing(df_PG_human['choices'], df_PG_human['choices'], lim_b=20)
Public_Goods_gpt4_result = simulate_Turing(df_PG_human['choices'], df_PG_gpt4['choices'], lim_b=20)
Public_Goods_turbo_result = simulate_Turing(df_PG_turbo['choices'], df_PG_turbo['choices'], lim_b=20)

Bomb_Risk_human_result = simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_human[''])
Bomb_Risk_gpt4_result = simulate_Turing(prefix_to_choices_human[''],prefix_to_choices_model['ChatGPT-4'][''])
Bomb_Risk_turbo_result = simulate_Turing(prefix_to_choices_human[''], prefix_to_choices_model['ChatGPT-3'][''])

Prisoner_Dilemma_human_result = r_coo_human * r_def_human, r_coo_human * r_coo_human + r_def_human * r_def_human, r_def_human * r_coo_human
Prisoner_Dilemma_gpt4_result = r_coo_human * r_def_gpt4, r_coo_human * r_coo_gpt4 + r_def_human * r_def_gpt4, r_def_human * r_coo_gpt4
Prisoner_Dilemma_turbo_result = r_coo_human * r_def_turbo, r_coo_human * r_coo_turbo + r_def_human * r_def_turbo, r_def_human * r_coo_turbo



100%|██████████| 100000/100000 [01:20<00:00, 1243.70it/s]


In [173]:

Average_human_result = tuple(sum(results) / 8 for results in zip(Dictator_human_result, Ultimatum_1_human_result, Ultimatum_2_human_result, Trust_1_human_result, Trust_3_human_result, Public_Goods_human_result, Bomb_Risk_human_result, Prisoner_Dilemma_human_result))
Average_gpt4_result = tuple(sum(results) / 8 for results in zip(Dictator_gpt4_result, Ultimatum_1_gpt4_result, Ultimatum_2_gpt4_result, Trust_1_gpt4_result, Trust_3_gpt4_result, Public_Goods_gpt4_result, Bomb_Risk_gpt4_result, Prisoner_Dilemma_gpt4_result))
Average_turbo_result = tuple(sum(results) / 8 for results in zip(Dictator_turbo_result, Ultimatum_1_turbo_result, Ultimatum_2_turbo_result, Trust_1_turbo_result, Trust_3_turbo_result, Public_Goods_turbo_result, Bomb_Risk_turbo_result, Prisoner_Dilemma_turbo_result))

data = {
    'Average': {
        'Human': Average_human_result,
        'GPT-4': Average_gpt4_result,
        'GPT-3': Average_turbo_result,
    },
    'Dictator': {
        'Human': Dictator_human_result,
        'GPT-4': Dictator_gpt4_result,
        'GPT-3': Dictator_turbo_result,
    },
    'Ultimatum 1': {
        'Human': Ultimatum_1_human_result,
        'GPT-4': Ultimatum_1_gpt4_result,
        'GPT-3': Ultimatum_1_turbo_result,
    },
    'Ultimatum 2': {
        'Human': Ultimatum_2_human_result,
        'GPT-4': Ultimatum_2_gpt4_result,
        'GPT-3': Ultimatum_2_turbo_result,
    },
    'Trust 1': {
        'Human': Trust_1_human_result,
        'GPT-4': Trust_1_gpt4_result,
        'GPT-3': Trust_1_turbo_result,
    },
    'Trust 3': {
        'Human': Trust_3_human_result,
        'GPT-4': Trust_3_gpt4_result,
        'GPT-3': Trust_3_turbo_result,
    },
    'Public Goods': {
        'Human': Public_Goods_human_result,
        'GPT-4': Public_Goods_gpt4_result,
        'GPT-3': Public_Goods_turbo_result,
    },
    'Bomb Risk': {
        'Human': Bomb_Risk_human_result,
        'GPT-4': Bomb_Risk_gpt4_result,
        'GPT-3': Bomb_Risk_turbo_result,
    },
    'Prisoner‘s Dilemma': {
        'Human': Prisoner_Dilemma_human_result,
        'GPT-4': Prisoner_Dilemma_gpt4_result,
        'GPT-3': Prisoner_Dilemma_turbo_result,
    }
}

In [174]:
# Transform the data into a format suitable for Altair
records = []
for scenario, scenario_data in data.items():
    for entity, values in scenario_data.items():
        records.append({
            "scenario": scenario,
            "entity": entity,
            "Estimated More likely Human": values[0],
            "Estimated Equally likely Human/AI": values[1],
            "Estimated More Likely AI": values[2]
        })

# Create a DataFrame
df = pd.DataFrame.from_records(records)

# Melt the DataFrame to have category names and percentages in separate columns
df_melted = df.melt(id_vars=['scenario', 'entity'], 
                    value_vars=['Estimated More likely Human', 'Estimated Equally likely Human/AI', 'Estimated More Likely AI'],
                    var_name='category', value_name='percentage')

# Define the color scale
color_scale = alt.Scale(domain=['Estimated More likely Human', 'Estimated Equally likely Human/AI', 'Estimated More Likely AI'],
                        range=['#2ca02c', '#ff7f0e', '#d62728'])

# Create the selection for the interactive highlight
highlight = alt.selection_multi(on='click',fields=['entity'], empty='none')
# alt.selection_single(on='mouseover', fields=['category'], empty='none')

# Define the base chart with a bar mark and the selection highlight
base_chart = alt.Chart(df_melted).mark_bar().encode(
    x=alt.X('sum(percentage)', stack="normalize", title='', axis=alt.Axis(format='.0%')),
    y=alt.Y('entity:N', title='', sort=alt.EncodingSortField('entity', op='min', order='descending')),
    color=alt.Color('category', scale=color_scale),
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.6)),
    tooltip=[alt.Tooltip('scenario'), alt.Tooltip('entity'), alt.Tooltip('category'), alt.Tooltip('sum(percentage):Q', format='.2%')]
).properties(
    width=450,  # Width of the individual charts
    height=200
).add_selection(
    highlight,
)

# Facet the base chart into two rows
faceted_chart = base_chart.facet(
    facet=alt.Facet('scenario', title='', header=alt.Header(labelOrient='top', titleOrient='top')),
    columns=3  # Display four graphs in each column
)

# Adjust spacing between the rows for better readability
faceted_chart = faceted_chart.configure_facet(spacing=10)

# Show the chart
faceted_chart

/Users/Boyan/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/Boyan/anaconda3/lib/python3.11/site-packages/altair/vegalite/v5/api.py:398: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
/Users/Boyan/anaconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.FacetChart(...)